In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import datetime,gc,math
import random
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
%matplotlib inline

In [ ]:
def active_time_transform(df):
    df['role_created_3_active_time'].fillna('"0-8,0","8-12,0","12-14,0","14-18,0","18-24,0"', inplace=True)
    df['role_created_3_active_time'] = df['role_created_3_active_time'].apply(lambda x:str(x)[1:-1])
    temp = df['role_created_3_active_time'].str.split(',',expand=True).iloc[:,[1,3,5,7,9]].rename(columns={1:'active_0-8',
                                        3:'active_8-12',5:'active_12-14',7:'active_14-18',9:'active_18-24'})
    for i in range(len(temp.columns)):
        temp.iloc[:,i] = temp.iloc[:,i].apply(lambda x:str(x)[:-1])
    df = df.join(temp).drop(['role_created_3_active_time'], axis=1)
    return df

def pay_grade_transform(df):
    df['role_created_3_pay_grade'].fillna('[0,0,0,0,0,0,0]',inplace=True)
    from ast import literal_eval
    df['role_created_3_pay_grade'] = df['role_created_3_pay_grade'].apply(literal_eval)
    df = df.join(df['role_created_3_pay_grade'].apply(pd.Series)).rename(columns={0:'pay_grade_1',
                1:'pay_grade_2',2:'pay_grade_3',3:'pay_grade_4',4:'pay_grade_5',5:'pay_grade_6',6:'pay_grade_7'}).drop(['role_created_3_pay_grade'], axis=1)
    return df

In [2]:
# %%time
role_info = pd.read_csv('./cp_role_info.csv')
# role_info = active_time_transform(role_info)
# role_info = pay_grade_transform(role_info)
# role_info.to_pickle('./role_info_1.pickle')

In [87]:
role_info = pd.read_pickle('./role_info_1.pickle')
# 删除2020-7-19日前30天的数据
role_info['create_role_time'] = pd.to_datetime(role_info['create_role_time'], format='%Y-%m-%d %H:%M:%S')
role_info.drop(role_info[role_info['create_role_time'] > (role_info['create_role_time'].max() + 
                                                          datetime.timedelta(-30))].index,inplace=True)

In [88]:
pd.set_option('display.max_columns',None)
role_info = role_info.drop_duplicates()
# print(role_info.shape)
# role_info.head(10)
role_info.drop(['role_id'], axis=1, inplace=True)
role_info.drop_duplicates(subset=['user_id','cp_server_no','cp_role_id'], inplace=True)
# 删除没有心跳数据的样本(1、没有登录;2、没有拿到心跳)
role_info.dropna(subset=['role_created_3_login_num'], inplace=True)
role_info.dropna(subset=['role_created_3_online'], inplace=True)
role_info['role_created_3_pay_num'].fillna(0, inplace=True)
role_info['role_created_3_pay_sum'].fillna(0, inplace=True)
role_info['role_created_3_active'] = role_info['role_created_3_active'].clip(0,3)
role_info['role_created_3_pay_rate'] = role_info['role_created_3_pay_num'] / (role_info['role_created_3_active'] + 1e-4)
role_info['role_created_3_pay_avg'] = role_info['role_created_3_pay_sum'] / (role_info['role_created_3_pay_num'] + 1e-4)

In [89]:
# 清洗设备金额
role_info.loc[role_info['p_model'] == 'iPhone 6s Plus','model_money'] = 3500
role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 5000
role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 2500
role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 2000
role_info['model_money'] = role_info['model_money'].replace(0, np.nan)
role_info['model_money'].fillna(role_info[role_info['model_money'] !=0]['model_money'].mean(),inplace=True)
bins=[0,1000,1500,2000,2500,3000,3500,4000,4500,5000,6000,8000,10000]
role_info['model_money_level'] = pd.cut(role_info['model_money'],bins,labels=False)

In [90]:
# 清洗login_num
# 截取3登陆次数大于200的样本
role_info['role_created_3_login_num'].clip(1,120,inplace=True)
tmp = role_info[role_info['role_created_3_login_num'] <2]
tmp['role_created_3_login_num'] = tmp['role_created_3_max_role_level'] * 0.6
role_info['role_created_3_login_num'].update(tmp['role_created_3_login_num'])
role_info['role_created_3_login_num'].clip(1,120,inplace=True)

In [91]:
# 清洗max_role_level
# 按前3天在线时长更新等级小于20级的
tmp = role_info[role_info['role_created_3_max_role_level'] <20]
tmp['role_created_3_max_role_level'] = tmp['role_created_3_online'] * 0.0011
role_info['role_created_3_max_role_level'].update(tmp['role_created_3_max_role_level'])
# 截取等级大于65级的
role_info['role_created_3_max_role_level'].clip(1,65,inplace=True)

In [92]:
# 清洗online
# 按等级补全在线时长
tmp = role_info[role_info['role_created_3_online'] <1800]
tmp['role_created_3_online'] = tmp['role_created_3_max_role_level'] * 907
role_info['role_created_3_online'].update(tmp['role_created_3_online'])
# 截断 最小5分钟，最大24小时
role_info['role_created_3_online'].clip(300,86400,inplace=True)

In [93]:
# 处理时间特征create_role_time
role_info = role_info.sort_values('create_role_time').reset_index(drop=True)
role_info['hour'] = role_info['create_role_time'].dt.hour
role_info['weekend'] = role_info['create_role_time'].dt.weekday
holidays = ['2019-09-13','2019-09-14','2019-09-15','2019-10-01','2019-10-02','2019-10-03','2019-10-04','2019-10-05','2019-10-06','2019-10-07',
            '2020-01-01','2020-01-24','2020-01-25','2020-01-26','2020-01-27','2020-01-28','2020-01-29','2020-01-30','2020-04-04','2020-04-05',
            '2020-04-06','2020-04-04','2020-05-01','2020-05-02','2020-05-03','2020-05-04','2020-05-05','2020-06-25','2020-06-27']
role_info['timestamp'] = role_info['create_role_time'].dt.date.apply(lambda x:str(x))
role_info['is_holidays'] = (role_info['timestamp'].isin(holidays)).astype(int)
role_info.drop(['timestamp'], axis=1, inplace=True)

In [94]:
# 合并role_user
user_info = pd.read_csv('./user_info.csv')
user_info.drop_duplicates(inplace=True)
data = pd.merge(role_info,user_info,on='user_id',how='left',validate='many_to_one')
# 标记用户时否绑定了手机，绑定;1    未绑定：0
data['mobile'] = pd.to_numeric(data['mobile'],errors='coerce')
data['mobile'] = data['mobile'].apply(lambda x:0 if np.isnan(x) == True else 1)
# 用户注册时间与角色注册时间的时间差,并进行分桶
data['reg_date'] = pd.to_datetime(data['reg_date'])
data['time_interval'] = (data['create_role_time'] - data['reg_date']).dt.total_seconds()/3600
data['time_interval'] = data['time_interval'].clip(0,2000)
data['time_interval'] = pd.cut(data['time_interval'], bins=[0,1,3,12,24,72,168,720,2000],labels=False)

data['active_0-8'] = pd.to_numeric(data['active_0-8'], errors='coerce')
data['active_8-12'] = pd.to_numeric(data['active_8-12'], errors='coerce')
data['active_12-14'] = pd.to_numeric(data['active_12-14'], errors='coerce')
data['active_14-18'] = pd.to_numeric(data['active_14-18'], errors='coerce')
data['active_18-24'] = pd.to_numeric(data['active_18-24'], errors='coerce')

In [95]:
data.to_pickle('./data.pickle')

In [96]:
gc.collect()

21216

In [97]:
data.head()

,user_id,cp_server_no,cp_role_id,create_role_time,p_model,sys_ver,model_money,role_created_3_login_num,role_created_3_active,role_created_3_online,role_created_3_max_role_level,role_created_3_ip_num,role_created_3_device_num,role_created_3_city_num,role_created_3_pay_num,role_created_3_pay_sum,role_created_30_pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_grade_7,role_created_3_pay_rate,role_created_3_pay_avg,model_money_level,hour,weekend,is_holidays,reg_date,mobile,user_type,user_flag,platform,channel_id,source_id,user_creates_3_server_num,user_creates_3_role_num,time_interval
0,90001783,660011,12927595,2019-08-16 13:46:01,OPPO A59s,22,690.000000,1.0,1.0,7288.0,8.0168,1.0,1.0,1.0,0.0,0.0,NaN,0,0,15,10,0,0,0,0,0,0,0,0,0.0,0.0,0.0,13,4,0,2019-08-16 13:46:00,0,18.0,1.0,1.0,6867.0,198844.0,1.0,2.0,0.0
1,90001801,660011,12927596,2019-08-16 13:46:17,OPPO R11st,25,2523.134967,1.0,1.0,907.0,1.0000,1.0,1.0,1.0,0.0,0.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,4.0,13,4,0,2019-08-16 13:46:17,0,18.0,1.0,1.0,6867.0,198844.0,1.0,1.0,NaN
2,89947274,660011,12927597,2019-08-16 13:46:30,PE-TL00M,4.4.2,2523.134967,3.6,1.0,907.0,1.0000,1.0,1.0,1.0,0.0,0.0,NaN,0,0,2,0,0,0,0,0,0,0,0,0,0.0,0.0,4.0,13,4,0,2019-08-15 20:59:33,0,1.0,1.0,1.0,20762.0,199152.0,2.0,2.0,3.0
3,90001815,660011,12927598,2019-08-16 13:46:34,Redmi Note 7 Pro,9,1207.630000,1.0,1.0,907.0,1.0000,1.0,1.0,1.0,0.0,0.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,13,4,0,2019-08-16 13:46:26,0,1.0,1.0,1.0,6770.0,188588.0,1.0,1.0,0.0
4,90001842,660011,12927599,2019-08-16 13:46:46,OPPO R9km,5.1,2499.000000,10.8,1.0,2437.0,2.6807,1.0,1.0,1.0,0.0,0.0,NaN,0,0,9,0,0,0,0,0,0,0,0,0,0.0,0.0,3.0,13,4,0,2019-08-16 13:46:41,0,1.0,1.0,1.0,6770.0,191645.0,1.0,1.0,0.0
